Dưới đây là một bộ kiểm thử đơn giản cho class `ThyroidCancerDataset` mà bạn đã cung cấp. Bộ kiểm thử sẽ đảm bảo rằng các chức năng chính của class hoạt động đúng.

### Các bước thực hiện:

1. **Kiểm tra việc khởi tạo class và thu thập đường dẫn ảnh cùng nhãn.**
2. **Kiểm tra việc cân bằng lớp chỉ trong tập huấn luyện.**
3. **Kiểm tra phương thức `__getitem__`.**

### Giải thích các bước kiểm thử:

1. **`setUp` và `tearDown`:**
   - Tạo một thư mục tạm thời với cấu trúc tương tự như dataset và tạo các ảnh giả để kiểm thử.
   - Sau khi kiểm thử xong, thư mục tạm thời sẽ được dọn dẹp.

2. **`test_init`:**
   - Kiểm tra việc khởi tạo class `ThyroidCancerDataset` với các tham số mặc định.
   - Đảm bảo rằng số lượng ảnh và nhãn được thu thập đúng.

3. **`test_balance_classes`:**
   - Kiểm tra việc cân bằng lớp bằng cách khởi tạo class với tham số `balance=True`.
   - Đảm bảo rằng số lượng ảnh sau khi cân bằng là đúng.

4. **`test_getitem`:**
   - Kiểm tra phương thức `__getitem__` để đảm bảo rằng ảnh và nhãn được trả về đúng loại dữ liệu.

### Lưu ý:

- Để chạy bộ kiểm thử này, bạn cần chắc chắn rằng các thư viện cần thiết (như `unittest`, `torchvision`, `PIL`) đã được cài đặt.
- Trong thực tế, bạn có thể cần thêm nhiều kiểm thử hơn để kiểm tra các trường hợp khác nhau và đảm bảo tính toàn vẹn của dataset.

In [1]:
%pip install unittest

ERROR: Could not find a version that satisfies the requirement unittest (from versions: none)
ERROR: No matching distribution found for unittest
Note: you may need to restart the kernel to use updated packages.


In [4]:
%cd /mnt/Data/Projects/ThyroidCancerClassifier

/mnt/Data/Projects/ThyroidCancerClassifier


/home/harito/venv/pyvenv311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import unittest
from torchvision import transforms
import os
import tempfile
from PIL import Image
import numpy as np
from src.data_preparation.ThyroidCancerDataLoader import ThyroidCancerDataset

class TestThyroidCancerDataset(unittest.TestCase):

    def setUp(self):
        # Create a temporary directory with a structure similar to the dataset
        self.test_dir = tempfile.TemporaryDirectory()
        os.makedirs(os.path.join(self.test_dir.name, 'B2'))
        os.makedirs(os.path.join(self.test_dir.name, 'B5'))
        os.makedirs(os.path.join(self.test_dir.name, 'B6'))

        # Create dummy images
        for subdir in ['B2', 'B5', 'B6']:
            for i in range(3):  # 3 images per class
                img = Image.new('RGB', (100, 100))
                img.save(os.path.join(self.test_dir.name, subdir, f'{subdir}_{i}.jpg'))

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomRotation(180),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])

    def tearDown(self):
        self.test_dir.cleanup()

    def test_init(self):
        dataset = ThyroidCancerDataset(
            img_dir=self.test_dir.name,
            transform=self.transform,
            classes={0: ["B2"], 1: ["B5"], 2: ["B6"]},
            balance=False,
            mode='train'
        )
        self.assertEqual(len(dataset), 9)
        self.assertEqual(len(dataset.img_paths), 9)
        self.assertEqual(len(dataset.labels), 9)
        self.assertIn(0, dataset.labels)
        self.assertIn(1, dataset.labels)
        self.assertIn(2, dataset.labels)

    def test_balance_classes(self):
        dataset = ThyroidCancerDataset(
            img_dir=self.test_dir.name,
            transform=self.transform,
            classes={0: ["B2"], 1: ["B5"], 2: ["B6"]},
            balance=True,
            mode='train'
        )
        self.assertEqual(len(dataset), 9)  # There should be 3*3 = 9 images after balancing

    def test_getitem(self):
        dataset = ThyroidCancerDataset(
            img_dir=self.test_dir.name,
            transform=self.transform,
            classes={0: ["B2"], 1: ["B5"], 2: ["B6"]},
            balance=False,
            mode='train'
        )
        img, label = dataset[0]
        self.assertIsInstance(img, torch.Tensor)
        self.assertIsInstance(label, int)

if __name__ == '__main__':
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/home/harito/.local/share/jupyter/runtime/kernel-v2-2280cda71NFDX1jH.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'